In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem1234561"}
# %mpy -s {"/dev/cu.usbmodem101"}
%mpy -s {"/dev/cu.usbmodem1101"}

In [ ]:
!mpremote reset

In [ ]:
# %%micropython
import time
from config import config
from secrets import secrets
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

TIMEOUT_MSEC = 5000

rtc = RTC()
wifi = Network.create_wifi()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport

# Work-around: disabling the power-saving mode on the STA_IF interface
epn.wifi._sta.config(pm=epn.wifi._sta.PM_NONE)

wifi.connect()
miniot_network.connect()
wifi.ntp_time_sync()
wifi.disconnect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")
print(f"peers: {epn.epn.get_peers()}")
print(f"Active?: {epn.wifi._sta.active()}")

while True:
    rxed_packets = []
    data_available = False

    # Receive data
    try:
        data_available = miniot_network.receive(rxed_packets)
    except Exception:
        data_available = False


In [ ]:
# %%micropython
import network
import time
from mp_libs.collections import defaultdict

def scan_channels():
    sta = network.WLAN(network.WLAN.IF_STA)
    sta.active(True)
    time.sleep(1)

    print("Scanning...")
    results = sta.scan()

    channel_rssi = defaultdict(list)
    for ssid, _, channel, rssi, _, _ in results:
        channel_rssi[channel].append(rssi)

    print("\nChannel | Count | Max RSSI | Avg RSSI")
    print("--------------------------------------")
    for channel in sorted(channel_rssi):
        rssi = channel_rssi[channel]
        rssi_count = len(rssi)
        rssi_max = max(rssi)
        rssi_ave = rssi_max / rssi_count
        print("   {:<5} | {:<5} | {:<9} | {:.1f}".format(channel, rssi_count, rssi_max, rssi_ave))


def scan_channels_congestion():
    sta = network.WLAN(network.STA_IF)
    sta.active(True)
    time.sleep(1)  # Allow Wi-Fi interface to stabilize

    print("Scanning for Wi-Fi networks...")
    results = sta.scan()

    # Group RSSI values by channel
    channel_stats = defaultdict(list)
    for ssid, bssid, channel, rssi, authmode, hidden in results:
        channel_stats[channel].append(rssi)

    print("\nChannel | SSIDs | Max RSSI | Avg RSSI")
    print("--------------------------------------")
    for channel in sorted(channel_stats):
        rssi_list = channel_stats[channel]
        ssid_count = len(rssi_list)
        max_rssi = max(rssi_list)
        avg_rssi = sum(rssi_list) / ssid_count
        print(f"{channel:^7} | {ssid_count:^5} | {max_rssi:^9} | {avg_rssi:.1f}")

    print("\nLook for channels with:")
    print("- Fewest SSIDs (less crowded)")
    print("- Lower RSSI values (weaker interference)")

# scan_channels()
scan_channels_congestion()



In [ ]:
!mpremote reset